# VID Path Pruner and Amplitude Calculator

Implements admissible path pruning and amplitude computation as per review 3.4.

In [ ]:
from collections import deque
import heapq
import numpy as np
import networkx as nx

# Graph and L_plus
G = nx.dodecahedral_graph()
L = nx.laplacian_matrix(G).toarray().astype(float)
eigvals, eigvecs = np.linalg.eigh(L)
eigvals[0] = 1.0
L_plus = eigvecs @ np.diag(1.0/eigvals) @ eigvecs.T

# Toy H_N
H_N = np.abs(L_plus)

# Coherence
phi = np.random.randn(20) * 0.4
kappa = 0.8
G_edge = {}
for u,v in G.edges():
    G_edge[(u,v)] = kappa * (phi[v] - phi[u])
    G_edge[(v,u)] = -G_edge[(u,v)]

def coherence_weight(path):
    return sum(G_edge.get((path[i],path[i+1]), 0) for i in range(len(path)-1))

def AVID(path, eps=1e-10):
    if len(path) > 15: return 0.0
    C = coherence_weight(path)
    if abs(C) > 4.0: return 0.0
    i, j = path[0], path[-1]
    AD = L_plus[i,i] + L_plus[j,j] - 2*L_plus[i,j]
    A_inf = np.abs(H_N[i,j])
    return AD * np.exp(-C) * A_inf

def compute_amplitude(start, target, max_paths=100000):
    heap = []
    visited = set()
    total = 0.0
    count = 0
    heapq.heappush(heap, (-1.0, 0, [start]))
    while heap and count < max_paths:
        neg_w, length, path = heapq.heappop(heap)
        node = path[-1]
        if node == target:
            contrib = AVID(path)
            total += contrib
            count += 1
        if length >= 12: continue
        for nbr in G.neighbors(node):
            if nbr in path[-3:]: continue
            new_path = path + [nbr]
            key = tuple(new_path)
            if key in visited: continue
            visited.add(key)
            est = AVID(new_path[:3] + [target])
            if est > 1e-14:
                heapq.heappush(heap, (-est, length+1, new_path))
    return total

A = compute_amplitude(0, 10)
print(f'Amplitude A(0→10) ≈ {A:.6e}')